In [11]:
import numpy  as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F

In [12]:
# Build CustomedDataset
# Separate the given dataset into two parts, four for train and one for test

class CustomedDataSet(Dataset):
    def __init__(self, train=True):
        self.train = train
        if self.train :
            trainX = pd.read_csv('./train.csv',nrows=33600)
            trainY = trainX.label.as_matrix().tolist()
            trainX = trainX.drop('label',axis=1).as_matrix().reshape(trainX.shape[0], 1, 28, 28)
            self.datalist = trainX
            self.labellist = trainY
        else:
            testX = pd.read_csv('./train.csv')
            testX = testX.drop(testX.head(33600).index)
            testY = testX.label.as_matrix().tolist()
            testX = testX.drop('label',axis=1).as_matrix().reshape(testX.shape[0], 1, 28, 28)
            self.datalist = testX
            self.labellist = testY
            
    def __getitem__(self, index):
        if self.train:
            return torch.Tensor(self.datalist[index].astype(float)),self.labellist[index]
        else:
            return torch.Tensor(self.datalist[index].astype(float)),self.labellist[index]
    
    def __len__(self):
        return self.datalist.shape[0]

In [13]:
batch_size = 100

In [14]:
train_dataset = CustomedDataSet()
test_dataset  = CustomedDataSet(train=False)
train_loader  = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,num_workers=2)
test_loader   = DataLoader(dataset= test_dataset,batch_size=batch_size,shuffle=False)

/home/peng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
/home/peng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
/home/peng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/home/peng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [72]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.features = nn.Sequential( 
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 8, stride = 2, padding = 1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(in_channels = 64, out_channels = 192, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(in_channels = 192, out_channels = 384, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 2, stride =2),
            
            nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )   
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256*3*3,1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024,64),
            nn.ReLU(inplace = True),
            nn.Linear(64,10),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256*3*3)
        x = self.classifier(x)
        return x

In [73]:
alexnet = AlexNet()

In [74]:
# Hyper Parameters

num_epochs = 5
learning_rate = 0.001

In [75]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alexnet.parameters(),lr=learning_rate)

In [76]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = alexnet(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 112 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' %(epoch+1, num_epochs, i+1, 
                                                              len(train_dataset)//batch_size, loss.item()))

Epoch [1/5], Iter [112/336] Loss: 0.4001
Epoch [1/5], Iter [224/336] Loss: 0.1658
Epoch [1/5], Iter [336/336] Loss: 0.1159
Epoch [2/5], Iter [112/336] Loss: 0.1083
Epoch [2/5], Iter [224/336] Loss: 0.0390
Epoch [2/5], Iter [336/336] Loss: 0.0256
Epoch [3/5], Iter [112/336] Loss: 0.0987
Epoch [3/5], Iter [224/336] Loss: 0.0712
Epoch [3/5], Iter [336/336] Loss: 0.0919
Epoch [4/5], Iter [112/336] Loss: 0.0152
Epoch [4/5], Iter [224/336] Loss: 0.0143
Epoch [4/5], Iter [336/336] Loss: 0.0682
Epoch [5/5], Iter [112/336] Loss: 0.0417
Epoch [5/5], Iter [224/336] Loss: 0.0162
Epoch [5/5], Iter [336/336] Loss: 0.0488


In [77]:
alexnet.eval() 

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(8, 8), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=2304, out_features=1024, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=1024, out_features=64, bias=True)
    (5): ReLU(inplace)
    (6): Linear(in_features=64, out_features=10

In [78]:
ans = torch.LongTensor()    # build a tensor to concatenate answers

In [79]:
for (images,labels) in test_loader:
    images = Variable(images)
    outputs = alexnet(images)
    _,predicted = torch.max(outputs.data, 1)
    ans = torch.cat((ans,predicted),0)

In [80]:
ans = ans.numpy()

In [81]:
corr=0
for i,(images,labels) in enumerate(test_dataset):
    if ans[i]==labels:
        corr=corr+1
print("%.3F"%(corr/84))


98.821
